In [40]:
# Working
import pandas as pd
from datetime import datetime, timedelta

# data = [
#   #   {
#   #   "EVENT_NO_TRIP": 229438098,
#   #   "EVENT_NO_STOP": 229438136,
#   #   "OPD_DATE": "30DEC2022:00:00:00",
#   #   "VEHICLE_ID": 3226,
#   #   "METERS": 140605,
#   #   "ACT_TIME": 37013,
#   #   "GPS_LONGITUDE": -122.779442,
#   #   "GPS_LATITUDE": 45.42606,
#   #   "GPS_SATELLITES": 11.0,
#   #   "GPS_HDOP": 0.9
#   # },
#   # {
#   #   "EVENT_NO_TRIP": 229438098,
#   #   "EVENT_NO_STOP": 229438136,
#   #   "OPD_DATE": "30DEC2022:00:00:00",
#   #   "VEHICLE_ID": 3226,
#   #   "METERS": 140657,
#   #   "ACT_TIME": 37018,
#   #   "GPS_LONGITUDE": -122.77895,
#   #   "GPS_LATITUDE": 45.426392,
#   #   "GPS_SATELLITES": 11.0,
#   #   "GPS_HDOP": 0.9
#   # },
#     {
#         "EVENT_NO_TRIP": "215698125",
#         "EVENT_NO_STOP": "215698147",
#         "OPD_DATE": "10-Dec-22",
#         "VEHICLE_ID": "3022",
#         "METERS": "15957",
#         "ACT_TIME": "22896",
#         "VELOCITY": "-122.867627",
#         "DIRECTION": "45.51346",
#         "RADIO_QUALITY": "12.0",
#         "GPS_LONGITUDE": "0.7",
#     },
#     {
#         "EVENT_NO_TRIP": "215698125",
#         "EVENT_NO_STOP": "215698147",
#         "OPD_DATE": "11-Dec-22",
#         "VEHICLE_ID": "3022",
#         "METERS": "16030",
#         "ACT_TIME": "22901",
#         "VELOCITY": "-122.867622",
#         "DIRECTION": "45.512805",
#         "RADIO_QUALITY": "12.0",
#         "GPS_LONGITUDE": "0.7",
#     },
# ]

# data = pd.read_json(
#     "/Users/vysalikallepalli/Desktop/CS510_DataEng_Project/2023-05-08.json"
# )
# data = pd.read_json(
#     '/Users/vysalikallepalli/Desktop/CS510_DataEng_Project/2023-04-16.json')
data = pd.read_json(
    "/Users/vysalikallepalli/Desktop/CS510_DataEng_Project/2023-05-10.json"
)


df = pd.DataFrame(data)


def data_transform(df):
    if df["OPD_DATE"].str.contains(r"\d{2}-[A-Za-z]{3}-\d{2}").any():
        filtered_df = df.copy()
        filtered_df.rename(
            columns={
                "EVENT_NO_TRIP": "trip_id",
                "OPD_DATE": "tstamp",
                "VELOCITY": "longitude",
                "DIRECTION": "latitude",
                "RADIO_QUALITY": "gps_satellites",
                "GPS_LONGITUDE": "gps_hdop",
            },
            inplace=True,
        )
        filtered_df.columns = filtered_df.columns.str.lower()
    else:
        filtered_df = df.copy()
        filtered_df.rename(
            columns={
                "EVENT_NO_TRIP": "trip_id",
                "OPD_DATE": "tstamp",
                "GPS_LONGITUDE": "longitude",
                "GPS_LATITUDE": "latitude",
            },
            inplace=True,
        )
        filtered_df.columns = filtered_df.columns.str.lower()

    filtered_df["tstamp"] = filtered_df["tstamp"].apply(
        lambda value: pd.to_datetime(value, format="%d-%b-%y", errors="coerce")
        if len(value) <= 11
        else pd.to_datetime(value, format="%d%b%Y:%H:%M:%S", errors="coerce")
    )
    filtered_df["act_time"] = pd.to_numeric(filtered_df["act_time"], errors="coerce")
    filtered_df["tstamp"] = filtered_df.apply(
        lambda row: row["tstamp"] + timedelta(seconds=row["act_time"])
        if pd.notnull(row["tstamp"])
        else "",
        axis=1,
    )
    filtered_df = filtered_df.sort_values(["trip_id", "tstamp"])
    filtered_df["dmeters"] = filtered_df.groupby(["trip_id"])["meters"].diff()
    filtered_df["dtimestamp"] = filtered_df.groupby(["trip_id"])["tstamp"].diff()
    filtered_df["speed"] = filtered_df.apply(
        lambda row: round(row["dmeters"] / row["dtimestamp"].total_seconds(), 2), axis=1
    )
    filtered_df["speed"] = filtered_df.groupby(["trip_id"])["speed"].fillna(
        method="bfill"
    )
    filtered_df["service_key"] = filtered_df["tstamp"].dt.dayofweek.apply(
        lambda day: "Weekday" if day < 5 else ("Saturday" if day == 5 else "Sunday")
    )
    # filtered_df = filtered_df.drop(
    #     ["dmeters", "dtimestamp", "gps_satellites", "gps_hdop", "event_no_stop"], axis=1
    # )
    return filtered_df


transformed_df = data_transform(df)
# display(transformed_df.style.hide_index())
display(transformed_df)

,trip_id,event_no_stop,tstamp,vehicle_id,meters,act_time,longitude,latitude,gps_satellites,gps_hdop,dmeters,dtimestamp,speed,service_key
201884,229810809,229810810,2023-01-01 07:25:53,4303,51,26753,-122.843775,45.504427,12.0,0.8,NaN,NaT,3.40,Sunday
201885,229810809,229810810,2023-01-01 07:25:58,4303,68,26758,-122.843660,45.504552,12.0,0.8,17.0,0 days 00:00:05,3.40,Sunday
201886,229810809,229810812,2023-01-01 07:26:08,4303,72,26768,-122.843652,45.504592,12.0,0.8,4.0,0 days 00:00:10,0.40,Sunday
201887,229810809,229810812,2023-01-01 07:26:13,4303,99,26773,-122.843587,45.504833,12.0,0.7,27.0,0 days 00:00:05,5.40,Sunday
201888,229810809,229810812,2023-01-01 07:26:18,4303,128,26778,-122.843480,45.505092,12.0,0.8,29.0,0 days 00:00:05,5.80,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212941,231115541,231115550,2023-01-02 01:44:52,4057,338798,92692,-122.646862,45.493935,12.0,0.7,9.0,0 days 00:00:05,1.80,Weekday
212942,231115541,231115550,2023-01-02 01:44:57,4057,338806,92697,-122.646828,45.493858,12.0,0.7,8.0,0 days 00:00:05,1.60,Weekday
212943,231115541,231115551,2023-01-02 01:45:32,4057,338807,92732,-122.646828,45.493838,12.0,0.7,1.0,0 days 00:00:35,0.03,Weekday
212944,231115541,231115551,2023-01-02 01:45:37,4057,338816,92737,-122.646838,45.493763,12.0,1.0,9.0,0 days 00:00:05,1.80,Weekday


In [41]:
# transformed_df.to_csv("2023-05-08_transformed_df.csv")

In [56]:
df1 = pd.read_json(
    "/Users/vysalikallepalli/Desktop/CS510_DataEng_Project/2023-05-10.json"
)
# print(type[df1['VEHICLE_ID']])


def validate(df1):
    assertion1 = "The vehicle_id for every bus is a positive number"
    # id = df1["VEHICLE_ID"].astype(int)
    assert (df1["VEHICLE_ID"].all()) > 0, f"{assertion1} NOT MET"
    print(f"{assertion1} MET")

    assertion2 = "For all records, there exists an EVENT_NO_TRIP which is a positive nine-digit number"
    assert df1["EVENT_NO_TRIP"].between(99999999, 10000000000).all(), print(
        f"{assertion2} NOT MET"
    )
    print(f"{assertion2} MET")

    assertion3 = "For all records, there exists an EVENT_NO_STOP which is a positive nine-digit number"
    assert df1["EVENT_NO_STOP"].between(99999999, 10000000000).all(), print(
        f"{assertion3} NOT MET"
    )
    print(f"{assertion3} MET")

    assertion4 = "The ACT_TIME column exists and represent seconds elapsed from midnight to the next day"
    act_time = df1["ACT_TIME"].astype(int)
    total_seconds = (24 * 60 * 60) + (60 * 60 * 4)
    if "ACT_TIME" in df1.columns:
        assert act_time.between(0, total_seconds).all(), print(
            f"{assertion4} NOT MET")
        print(f"{assertion4}  MET")

    assertion5 = "OPD_DATE format is either DD-MMM-YY or DDMMMYY:00:00:00"
    opd_format = df1["OPD_DATE"]
    pattern1 = r"\d{2}-[A-Za-z]{3}-\d{2}"
    pattern2 = r"\d{2}[A-Za-z]{3}\d{4}:\d{2}:\d{2}:\d{2}"
    if opd_format.str.contains(pattern1).any() or opd_format.str.contains(pattern2).any():
        print(f"{assertion5} MET")
    else:
        print(f"{assertion5} NOT MET")


df1 = validate(df1)

The vehicle_id for every bus is a positive number MET
For all records, there exists an EVENT_NO_TRIP which is a positive nine-digit number MET
For all records, there exists an EVENT_NO_STOP which is a positive nine-digit number MET
The ACT_TIME column exists and represent seconds elapsed from midnight to the next day  MET
OPD_DATE format is either DD-MMM-YY or DDMMMYY:00:00:00 MET


In [43]:
transformed_df.columns.tolist()

['trip_id',
 'event_no_stop',
 'tstamp',
 'vehicle_id',
 'meters',
 'act_time',
 'longitude',
 'latitude',
 'gps_satellites',
 'gps_hdop',
 'dmeters',
 'dtimestamp',
 'speed',
 'service_key']

In [44]:
print(type(transformed_df["speed"]))

<class 'pandas.core.series.Series'>


In [45]:
def validate_new(transformed_df):
    assertion6 = "In all records, the speed value is a non-negative number"
    if transformed_df["speed"].dtype == object:
        df["speed"] = transformed_df["speed"].astype(float)
        assert df["speed"].all() >= 0, print(f"{assertion6}: NOT MET")
    print(f"{assertion6}: MET")

    assertion7 = "The speed of a TriMet bus should not exceed 100 miles per hour"
    if transformed_df["speed"].dtype == object:
        new_df = transformed_df["speed"].astype(float)
        speed_mph = new_df["speed"] * 2.23694
        speed_limit = 100
        assert not (speed_mph > speed_limit).any(), f"{assertion7}: NOT MET"
    print(f"{assertion7}: MET")

    assertion8 = "If a latitude coordinate is present for every trip, then a corresponding longitude coordinate should also be present"
    assert (transformed_df["latitude"].notnull(
    ) == transformed_df["longitude"].notnull()).all(), f"{assertion8}: NOT MET"
    print(f"{assertion8}: MET")

    assertion9 = "gps_satellites exists and is a positive number"
    assert 'gps_satellites' in transformed_df.columns and transformed_df["gps_satellites"].all(
    ) >= 0, f"{assertion9}: NOT MET"
    print(f"{assertion9}: MET")

    assertion10 = "gps_hdop exists and is a positive number"
    assert 'gps_hdop' in transformed_df.columns and transformed_df["gps_hdop"].all(
    ) >= 0, f"{assertion10}: NOT MET"
    print(f"{assertion10}: MET")


df = validate_new(transformed_df)

In all records, the speed value is a non-negative number: MET
The speed of a TriMet bus should not exceed 100 miles per hour: MET
If a latitude coordinate is present for every trip, then a corresponding longitude coordinate should also be present: MET
gps_satellites exists and is a positive number: MET
gps_hdop exists and is a positive number: MET


In [46]:
display(transformed_df)

,trip_id,event_no_stop,tstamp,vehicle_id,meters,act_time,longitude,latitude,gps_satellites,gps_hdop,dmeters,dtimestamp,speed,service_key
201884,229810809,229810810,2023-01-01 07:25:53,4303,51,26753,-122.843775,45.504427,12.0,0.8,NaN,NaT,3.40,Sunday
201885,229810809,229810810,2023-01-01 07:25:58,4303,68,26758,-122.843660,45.504552,12.0,0.8,17.0,0 days 00:00:05,3.40,Sunday
201886,229810809,229810812,2023-01-01 07:26:08,4303,72,26768,-122.843652,45.504592,12.0,0.8,4.0,0 days 00:00:10,0.40,Sunday
201887,229810809,229810812,2023-01-01 07:26:13,4303,99,26773,-122.843587,45.504833,12.0,0.7,27.0,0 days 00:00:05,5.40,Sunday
201888,229810809,229810812,2023-01-01 07:26:18,4303,128,26778,-122.843480,45.505092,12.0,0.8,29.0,0 days 00:00:05,5.80,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212941,231115541,231115550,2023-01-02 01:44:52,4057,338798,92692,-122.646862,45.493935,12.0,0.7,9.0,0 days 00:00:05,1.80,Weekday
212942,231115541,231115550,2023-01-02 01:44:57,4057,338806,92697,-122.646828,45.493858,12.0,0.7,8.0,0 days 00:00:05,1.60,Weekday
212943,231115541,231115551,2023-01-02 01:45:32,4057,338807,92732,-122.646828,45.493838,12.0,0.7,1.0,0 days 00:00:35,0.03,Weekday
212944,231115541,231115551,2023-01-02 01:45:37,4057,338816,92737,-122.646838,45.493763,12.0,1.0,9.0,0 days 00:00:05,1.80,Weekday


In [47]:

data = [
    {
        "EVENT_NO_TRIP": 229438098,
        "EVENT_NO_STOP": 229438136,
        "OPD_DATE": "30DEC2022:00:00:00",
        "VEHICLE_ID": 3226,
        "METERS": 140605,
        "ACT_TIME": 37013,
        "GPS_LONGITUDE": -122.779442,
        "GPS_LATITUDE": 45.42606,
        "GPS_SATELLITES": 11.0,
        "GPS_HDOP": 0.9
    },
    {
        "EVENT_NO_TRIP": 229438098,
        "EVENT_NO_STOP": 229438136,
        "OPD_DATE": "30DEC2022:00:00:00",
        "VEHICLE_ID": 3226,
        "METERS": 140657,
        "ACT_TIME": 37018,
        "GPS_LONGITUDE": -122.77895,
        "GPS_LATITUDE": 45.426392,
        "GPS_SATELLITES": 11.0,
        "GPS_HDOP": 0.9
    },
    {
        "EVENT_NO_TRIP": "215698125",
        "EVENT_NO_STOP": "215698147",
        "OPD_DATE": "10-Dec-22",
        "VEHICLE_ID": "3022",
        "METERS": "15957",
        "ACT_TIME": "22896",
        "VELOCITY": "-122.867627",
        "DIRECTION": "45.51346",
        "RADIO_QUALITY": "12.0",
        "GPS_LONGITUDE": "0.7",
    },
    {
        "EVENT_NO_TRIP": "215698125",
        "EVENT_NO_STOP": "215698147",
        "OPD_DATE": "11-Dec-22",
        "VEHICLE_ID": "3022",
        "METERS": "16030",
        "ACT_TIME": "22901",
        "VELOCITY": "-122.867622",
        "DIRECTION": "45.512805",
        "RADIO_QUALITY": "12.0",
        "GPS_LONGITUDE": "0.7",
    },
]


type(data)

list